# Firmware para <span style="color:#F37263">Arduino M1</span> (Comunicación serie entre PC y Arduino)

La siguiente notebook es parte del firmware de la versión *FMR-001 Rev A* del <span style="color:#F37263">**Arduino M1**</span>.

Sirve para,
- Comunicarse con la PC que procesa y clasifica la señal de EEG.
- Enviar los comandos de movimiento y recibir el estado del robot desde <span style="color:#008a3e">**Arduino M3**</span>.

**NOTA:** El código presentado debajo será editado para gregarle funcionalidades.

### Declaración de variables globales

Lo primero que hacemos incluir algunas librerias y declarar variables globales. Recordar que las variables globales pueden ser accedidas desde cualquier parte del código. Por lo tanto debemos ser cuidadosos/as cuando estemos generando cambios en estas.

Veamos la primera parte del código.

```c++

/******************************************************************
            VERSIÓN FMR-001 Rev A
******************************************************************/

#include "definiciones.h"
#include "inicializaciones.h"
#include "funciones.h"

/******************************************************************
  Declaración de variables para comunicación
/******************************************************************/

char inBuffDataFromPC = 3;
unsigned char incDataFromPC[3]; //variable para almacenar datos provenientes de la PC
char bufferIndex = 0;
bool sendDataFlag = 0;
bool newMessage = false;

unsigned char internalStatus[4]; //variable para enviar datos a la PC
char internalStatusBuff = 4;

unsigned char outputDataToRobot[4]; //variable para enviar datos al robot
char buffOutDataRobotSize = 4;
char buffOutDataRobotIndex = 0;

unsigned char incDataFromRobot[4]; //variable para recibir datos del robot
char incDataFromRobotSize = 4;
char incDataFromRobotIndex = 0;

/******************************************************************
  Variables para el control de flujo de programa
******************************************************************/
char sessionState = 0; //Sesión sin iniciar
char LEDVerde = 12;
char LEDTesteo = 13; //led de testeo

/******************************************************************
  Declaración de variables para control de estímulos
******************************************************************/

int frecTimer = 5000; //en Hz. Frecuencia de interrupción del timer.

//estímulo izquierdo
char estimIzq = 11;
bool estimIzqON = 0;//Esado que define si el LED se apgará o prenderá.
int frecEstimIzq = 11;
int acumEstimIzq = 0;
const int estimIzqMaxValue = (1/float(frecEstimIzq))*frecTimer;

//estímulo derecho
char estimDer = 7;
bool estimDerON = 0;//Esado que define si el LED se apgará o prenderá.
int frecEstimDer = 11;
int acumEstimDer = 0;
const int estimDerMaxValue = (1/float(frecEstimDer))*frecTimer;

/*Implementar lo siguiente
//estímulo adelante
//estímulo atrás
*/

char stimuli = ON; //variable golbal para control de estímulos
int acumuladorStimuliON = 0;
int trialNumber = 1;

/******************************************************************
  Variables control de movimiento
******************************************************************/
char movimiento = 0; //Robot en STOP
```

### Función setup()

Como sabemos, la función *setup()* de Arduino nos permite configurar e inicializar variables, puertos, interrupciones, entre otras.

Se ejecuta **una sola vez** al iniciar el programa.

Para la versión *FMR-001 Rev A* lo implementado es,

```c++
//FUNCION SETUP
void setup()
{
  noInterrupts();//Deshabilito todas las interrupciones
  pinMode(estimIzq,OUTPUT);
  pinMode(estimDer,OUTPUT);
  pinMode(LEDVerde,OUTPUT);
  pinMode(LEDTesteo,OUTPUT);
  iniTimer0(frecTimer); //inicio timer 0
  Serial.begin(19200); //iniciamos comunicación serie

  interrupts();//Habilito las interrupciones
}
```

Podemos ver que se inicializan los pines de los estímulos como salida. Se invoca a la función *iniTimer0()* la cual esta dentro de la libreria *funciones.h*.

### Función loop()

Vemos que la función *loop()* no tiene nada dentro.

```c++
void loop(){}
```

### Interrupción UART: Función para recibir y enviar datos por la UART

Cada vez que nos llega un dato por la UART del micronctrolador se genera una interrupción, la cual luego de que el microcontrolador ejecuta su última instrucción en curso y dependiendo de las prioridades de interrupción seteadas, la siguiente instrucción ejecutará la función *serialEvent()*. Esta función nos permite recibir y enviar mensajes por la UART.

Su implementación es la siguiente.

```c++
void serialEvent()
{
if (Serial.available() > 0) 
  {
    char val = char(Serial.read()) - '0';
    checkMessage(val); //chequeamos mensaje entrante        
    for(int index = 0; index < internalStatusBuff; index++) //enviamos estado 
      {Serial.write(internalStatus[index]);}
      Serial.write("\n");
  }
};
```

Es **importante** notar que con cada nuevo byte entrante se invoca a la función *checkMessage()*. Veremos más adelante de que se trata la misma.

Por otro lado, una vez chequeado el mensaje se envía un set de bytes con el **estado interno** de nuestro Arduino a la PC. Como vemos, siempre que llega un nuevo Byte se envía el estado interno.

<span style="color:red">**Importante:**</span> La actualización del estado interno de nuestro <span style="color:#F37263">**Arduino M1**</span>, el envío de un comando al <span style="color:#008a3e">**Arduino M3**</span>, entre otras acciones, estarán actualizadas con la llegada un nuevo Byte desde la PC, es decir, con cada inerrupción de la UART.

### Chequeando mensaje proveniente de la PC

Como vimos, con cada nuevo byte entrante desde la PC invocamos a la función *checkMessage()*. Su implementación para la versión actual del firmware es,

```c++
void checkMessage(char val)
{
    
  incDataFromPC[bufferIndex] = val;
  switch(bufferIndex)
  {
    case 0:
      if (incDataFromPC[bufferIndex] == 1) sessionState = SESSION_RUNNING;
      else sessionState = SESSION_STOP;
      digitalWrite(LEDTesteo,0);
      //else sessionState = STOP;
      break;
    case 1:
      if (incDataFromPC[bufferIndex] == 1) stimuli = ON;
      else stimuli = OFF;
      //else sessionState = STOP;
      break;

    case 2: //indica hacia donde se debe mover el vehículo
      char comando = incDataFromPC[bufferIndex];
      sendCommand(comando); //Es mejor hacer sendCommand(incDataFromPC[bufferIndex])
      break;          
  }
    
  bufferIndex++;
  if (bufferIndex >= inBuffDataFromPC) bufferIndex = 0;
};
```

Como vemos, el byte *char val* pasado como parámetro es almacenado dentro del arreglo de chars llamado *incDataFromPC* en la posición *bufferIndex*. Esto es **importante** ya que cada posición dentro del arreglo *incDataFromPC* posee un rol importante, recordar del código de la clase *ArduinoCommunication* de Python que el mensaje enviado desde la PC al <span style="color:#F37263">**Arduino M1**</span> esta tiene la forma,


```python
    self.systemControl = [self.sessionStatus,
                         self.stimuliStatus,
                         self.moveOrder]
```

Considerando lo anterior, en el caso de que *bufferIndex* tome el valor *2* estaremos almacenando en *incDataFromPC* la orden de movimiento para enviar al robot. Por lo tanto se invoca la función *sendCommand()* la cual enviará el comando de movimiento al <span style="color:#008a3e">**Arduino M3**</span> a través del Bluettoth para que el <span style="color:#008a3e">**Arduino M3**</span> controle el vehículo.

**Nota:** La función *sendCommand()* aún no esta implementada.

### Interrupción timer0

Para la versión de este firmware se configuro el *timer0* para interrumpir cada *0.2 milisegunds* (5000Hz).

Es <span style="color:red">**sumamente importante**</span> que el *timer0* produzca una interrupción en la frecuencia correcta. Es decir, cualquier cambio en la frecuencia de interrupción debe reflejarse en la variable *frecTimer* y por supuesto, en los registros del *timer 0* del micro. La configuración de los registros del *timer 0* del microcontrolador pueden verse dentro de la función *iniTimer0()* dentro de la libreria *inicializaciones.h*.

Por otro lado, siempre es recomendable chequear con un osciloscopio si se ha configurado correctamente un timer, generando una señal cuadrada a la salida de algún pin del microcontrolador que cambie de estado alto/bajo con cada interrupción del timer. Luego se mide el tiempo en que la señal esta en estado alto (o bajo).

##### Acciones al producirse una interrupción

Cada *0.2 ms* se ejecuta la siguiente rutina de código,

```c++
ISR(TIMER0_COMPA_vect)//Rutina interrupción Timer0.
{
  if(sessionState == SESSION_RUNNING) stimuliControl(); //Si la sesión comenzó, empezamos a generar los estímulos  
  else
  {
    //apago estímulos
    digitalWrite(estimIzq,0);
    digitalWrite(estimDer,0);
    digitalWrite(LEDTesteo,1);
  }
};
```

Su implementación es muy facil, vemos que si *if(sessionState == SESSION_RUNNING)* entonces invocamos la función *stimuliControl()*. Si lo anterior no se cumple, es de esperar que la sesión este frenada y por lo tanto se apagan los estímulos.

### Controlando estímulos - Función *stimuliControl()*

Para el control de estímulos se implemento la función *stimuliControl()*.

```c++
void stimuliControl()
{
  acumuladorStimuliON++;
  switch(stimuli)
  {
    case ON:
    //control estímulo izquierdo
      if (++acumEstimIzq >= estimIzqMaxValue)
      {
        estimIzqON = !estimIzqON;
        digitalWrite(estimIzq,estimIzqON);
        acumEstimIzq = 0; 
      } 

    //control estímulo derecho
      if (++acumEstimDer >= estimDerMaxValue)
      {
        estimDerON = !estimDerON;
        digitalWrite(estimDer,estimDerON);
        acumEstimDer = 0; 
      } 
      break;

    case OFF:
      {
        //Apagamos estímulos y reiniciamos contadores
        digitalWrite(estimIzq,0);
        acumEstimIzq = 0;
        digitalWrite(estimDer,0);
        acumEstimDer = 0;
        trialNumber++; //Sumamos un nuevo trial
        acumuladorStimuliON = 0; //reiniciamos acumulador para temporizar cada trial
        acumEstimIzq = 0;
        acumEstimDer = 0;
      } 
      break;
  }
}
```

Es importante notar que:
- Si la variable *stimuli* es *ON* indica que estamos en la parte del trial que **debemos** estimular. Por lo tanto, los estímulos (LEDs) se prenderán y apagaran según la frecuencia con la que hayan sido configuradas.
- Si la variable *stimuli* es *OFF* estamos en la parte del trial que **no debemos** estimular. Por lo tanto, apagamos los estímulos y reiniciamos variables de control a sus valores iniciales.

### <span style="color:#3363b0">Comentarios finales </span> 

El código hasta aquí presentado corresponde a la versión de firmware *FMR-001 Rev A*. Este código deberá crecer y agregar funcionalidades, además, cada equipo tiene vehículos diferentes y cantidades de comandos diferentes, por lo tanto, el código de este firmware será diferente para cada caso, no obstante, la lógica del control de flujo y de procesos **es la misma** para cada equipo, lo que cambia es la cantidad de datos que se envían y reciben.